## About the dataset

AMES Housing Dataset by Dean De Cock (Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

## Housing Prices visualizations

In [ ]:
%pip install pandas matplotlib seaborn numpy scikit-learn scipy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import os
import requests
warnings.filterwarnings('ignore')
%matplotlib inline

thePath = "./"
theFile = 'train.csv'
theLink = "https://dse200.dev/Day3/train.csv"

if not os.path.exists(thePath + theFile):
    r = requests.get(theLink)
    with open(thePath + theFile, 'wb') as f:
        f.write(r.content)


### Data Exploration
- Visualize
- Find Missing Data
- Look For Correlations

In [ ]:
df = pd.read_csv(thePath + 'train.csv') # loading the ames data

#### Analysing Sale Price

In [ ]:
df['SalePrice'].describe()

In [ ]:
sns.distplot(df['SalePrice']);

We observe that the data
- Deviates from the normal distribution.
- Has appreciable positive skewness.
- Shows peakedness.

In [ ]:
print("Skewness: %f" % df['SalePrice'].skew())
print("Kurtosis: %f" % df['SalePrice'].kurt())

### Examples of Relations with Numerical Variables

In [ ]:
var = 'GrLivArea'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

We notice a linear relationship

In [ ]:
var = 'TotalBsmtSF'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

linear relation with a higher slope

### Examples of Relations with Categorical Variables

In [ ]:
var = 'OverallQual'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);


we see a positive correlation between these two variables

In [ ]:
var = 'YearBuilt'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);

Although not a strong one, we still see a positive correlation between these two variables as well

### Correlation Matrix

In [ ]:
df.head()

In [ ]:

var = 'LotArea' # 'YearBuilt' 
data = pd.concat([df['SalePrice'], df[var]], axis=1)
#corrmat = df.corr()
data = pd.concat([df['SalePrice'], df['LotFrontage'],df['YearBuilt'],df['LotArea']], axis=1)
corrmat = data.corr()
#f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

From this overview of all the realtions

### We pick "k" columns which are most correlated with Sale Price

In [ ]:
k = 10 
cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
cols

In [ ]:
cmvals = df[cols].values.T
cmvals

In [ ]:
cm = np.corrcoef(df[cols].values.T)
sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

'OverallQual', 'GrLivArea' and 'TotalBsmtSF' are strongly correlated with 'SalePrice'<br/>
'GarageCars' can be assumed to be dependent on 'GarageArea'. Hence we choose only 'GarageCars' since its correlation with 'SalePrice' is higher <br/>
'TotalBsmtSF' and '1stFloor' represent kinda the same thing so we pick one ('TotalBsmtSF')<br/>
'TotRmsAbvGrd' and 'GrLivArea' have a high correlation as expected.<br/>
TODO :  Time series analysis for 'YearBuilt'


### Feature Engineering for Housing Prices

Let's engineer features to improve predictive performance! The Ames Housing dataset is rich with opportunities.

**Why Feature Engineering Matters for Real Estate:**
- **Age/depreciation:** Houses lose/gain value over time
- **Size efficiency:** Price per square foot varies
- **Quality combinations:** Quality × Size interactions
- **Neighborhood context:** Relative pricing matters

In [ ]:
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df[cols], size = 2.5)
plt.show()

### Data Cleaning

#### Missing Data

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

columns with more than 80% of the data is missing are chosen to be deleted. Hence the set of variables (e.g. 'PoolQC', 'MiscFeature', 'Alley', etc. are chosen for deletion. Further, these features show nearly 0 correlation with 'Sale Price'

'GarageCars' will represent most info regarding garages, hence other 'GarageX' variables can be ignored.

For 'Electrical', we can either use another value to fill the missing value or drop the observation. In this case we drop the observation

In [ ]:
(missing_data[missing_data['Total'] > 1]).index

In [ ]:
#df = df.drop((missing_data[missing_data['Total'] > 1]).index,1)
#df = df.drop(df.loc[df['Electrical'].isnull()].index)

#### Outliers

In [ ]:
var = 'GrLivArea'
data = pd.concat([df['SalePrice'], df[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

The rightmost observations in 'GrLivArea' seem to be outliers <br/>
The topmost observations in 'SalePrice' seem to follow the trend hence we do not consider them as outliers

In [ ]:
df.sort_values(by = 'GrLivArea', ascending = False)[:2]
df = df.drop(df[df['Id'] == 1299].index)
df = df.drop(df[df['Id'] == 524].index)

#### Normality

In [ ]:
sns.distplot(df['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['SalePrice'], plot=plt)

<b>Normal Probability Plot : </b>The data are plotted against a theoretical normal distribution in such a way that the points should form an approximate straight line. Departures from this straight line indicate departures from normality.

in case of positive skewness, log transformations usually works well

In [ ]:
df['SalePrice'] = np.log(df['SalePrice'])

Replotting

In [ ]:
sns.distplot(df['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['SalePrice'], plot=plt)

Checking the col : GrLivArea

In [ ]:
sns.distplot(df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['GrLivArea'], plot=plt)

Applying similar transformation

In [ ]:
df['GrLivArea'] = np.log(df['GrLivArea'])

In [ ]:
sns.distplot(df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['GrLivArea'], plot=plt)

Checking for col : TotalBsmtSF

In [ ]:
sns.distplot(df['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df['TotalBsmtSF'], plot=plt)

Here we see postive skewness but quite a few of the points are 0

In [ ]:
df.loc[df['TotalBsmtSF']>0,'TotalBsmtSF'] = np.log(df['TotalBsmtSF'])

In [ ]:
sns.distplot(df[df['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df[df['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
s = pd.Series(list('abca'))

In [ ]:
s

#### Credits

https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python <br/>
https://www.kaggle.com/ekami66/detailed-exploratory-data-analysis-with-python/notebook

#### 1. Age and Remodeling Features

**Justification:** Age affects value differently:
- House age (years since built)
- Years since remodel
- Is newly built? (premium)
- Never remodeled? (penalty)

In [ ]:
# Reload fresh data for feature engineering
df_fe = pd.read_csv(thePath + 'train.csv')

# Current year (or use max year in dataset)
current_year = df_fe['YrSold'].max()

# Age features
df_fe['house_age'] = current_year - df_fe['YearBuilt']
df_fe['years_since_remodel'] = current_year - df_fe['YearRemodAdd']
df_fe['is_newly_built'] = (df_fe['house_age'] <= 2).astype(int)
df_fe['never_remodeled'] = (df_fe['YearBuilt'] == df_fe['YearRemodAdd']).astype(int)

print("Age features created:")
print(df_fe[['YearBuilt', 'YearRemodAdd', 'house_age', 'years_since_remodel', 
             'is_newly_built', 'never_remodeled']].head())

#### 2. Total Square Footage and Ratios

**Justification:** Total living space and efficiency metrics:
- Total square footage (above + basement)
- Price per square foot (efficiency)
- Lot coverage ratio (house size vs lot size)
- Basement percentage

In [ ]:
# Total square footage
df_fe['total_sqft'] = df_fe['GrLivArea'] + df_fe['TotalBsmtSF']
df_fe['total_bathrooms'] = df_fe['FullBath'] + 0.5 * df_fe['HalfBath']
df_fe['total_porch_sf'] = (df_fe['OpenPorchSF'] + df_fe['EnclosedPorch'] + 
                            df_fe['3SsnPorch'] + df_fe['ScreenPorch'])

# Efficiency ratios
df_fe['basement_ratio'] = df_fe['TotalBsmtSF'] / (df_fe['total_sqft'] + 1)
df_fe['lot_coverage_ratio'] = df_fe['total_sqft'] / (df_fe['LotArea'] + 1)

# Quality × Size interaction
df_fe['quality_x_sqft'] = df_fe['OverallQual'] * df_fe['total_sqft']

print("Size and ratio features created:")
print(df_fe[['GrLivArea', 'TotalBsmtSF', 'total_sqft', 'basement_ratio', 
             'lot_coverage_ratio', 'quality_x_sqft']].head())

#### 3. Neighborhood Context (Target Encoding)

**Justification:** Neighborhood drives pricing:
- Average sale price per neighborhood
- Price relative to neighborhood
- Neighborhood wealth indicators

In [ ]:
# Neighborhood aggregations
neighborhood_stats = df_fe.groupby('Neighborhood')['SalePrice'].agg(['mean', 'median', 'std']).reset_index()
neighborhood_stats.columns = ['Neighborhood', 'neighborhood_avg_price', 
                               'neighborhood_median_price', 'neighborhood_price_std']

df_fe = df_fe.merge(neighborhood_stats, on='Neighborhood', how='left')

# Relative pricing
df_fe['price_vs_neighborhood'] = df_fe['SalePrice'] / (df_fe['neighborhood_avg_price'] + 1)

# Wealthy neighborhood indicator
df_fe['is_wealthy_neighborhood'] = (df_fe['neighborhood_avg_price'] > 
                                     df_fe['neighborhood_avg_price'].quantile(0.75)).astype(int)

print("Neighborhood features created:")
print(df_fe[['Neighborhood', 'SalePrice', 'neighborhood_avg_price', 
             'price_vs_neighborhood', 'is_wealthy_neighborhood']].head())

#### 4. Quality and Condition Scores

**Justification:** Combine quality/condition ratings:
- Overall quality score (quality + condition)
- Has garage (binary)
- Has basement (binary)
- Has fireplace, pool, etc.

In [ ]:
# Combined quality score
df_fe['total_quality_score'] = df_fe['OverallQual'] + df_fe['OverallCond']

# Binary amenity features
df_fe['has_garage'] = (df_fe['GarageArea'] > 0).astype(int)
df_fe['has_basement'] = (df_fe['TotalBsmtSF'] > 0).astype(int)
df_fe['has_fireplace'] = (df_fe['Fireplaces'] > 0).astype(int)
df_fe['has_pool'] = (df_fe['PoolArea'] > 0).astype(int)
df_fe['has_2nd_floor'] = (df_fe['2ndFlrSF'] > 0).astype(int)

# High-end features count
df_fe['luxury_features_count'] = (
    df_fe['has_pool'] + 
    df_fe['has_fireplace'] + 
    (df_fe['OverallQual'] >= 8).astype(int) +
    (df_fe['GarageCars'] >= 3).astype(int)
)

print("Quality and amenity features created:")
print(df_fe[['OverallQual', 'OverallCond', 'total_quality_score', 'has_garage', 
             'has_fireplace', 'luxury_features_count']].head())

### Summary: Housing Price Feature Engineering

| Feature Type | Examples | Impact | Insight |
|-------------|----------|--------|---------|
| **Age** | house_age, years_since_remodel | High | Depreciation matters |
| **Size/Ratios** | total_sqft, quality×sqft | Very High | Interactions boost value |
| **Neighborhood** | Avg price, relative pricing | Very High | Location, location, location |
| **Quality/Amenities** | Quality scores, luxury count | High | Premium features add value |

**Key Observations:**
- The notebook already identified top correlations (OverallQual, GrLivArea, etc.)
- Our features ADD context and interactions
- Target encoding (neighborhood averages) is powerful but watch for overfitting!

Now these engineered features are ready for modeling!

In [ ]:
# Feature count summary
print(f"Original features: {df.shape[1]}")
print(f"After feature engineering: {df_fe.shape[1]}")
print(f"New features created: {df_fe.shape[1] - df.shape[1]}")
print(f"\nSample of new features:")
new_features = [col for col in df_fe.columns if col not in df.columns]
print(new_features)